In [1]:
%load_ext autoreload
%autoreload 2
import os
from pathlib import Path
import sys
import torch
from src.models import Resnet18, Resnet152
from src.dataset import FlowImageDataset
from src.trainer import train
from src.predictor import Predictor
from src.metrics import transl_error, rot_error, recall
from src.result_visualization import ResultVisualization
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
from load_data import load_data, load_meshes_o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
obj_diameter = 0.24
do_training = True
use_pretrained = True
model_type = 'resnet152'

In [3]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [4]:
flow_images_path = Path("../../output", "flow_images")
poses_path = Path("../../output", "dataset_rendered", "shoe")
if do_training:
    train_ds = FlowImageDataset(flow_image_path=flow_images_path / "train", poses_path=poses_path, subset="train")
    val_ds = FlowImageDataset(flow_image_path=flow_images_path / "train", poses_path=poses_path, subset="val")
test_ds = FlowImageDataset(flow_image_path=flow_images_path / "test", poses_path=poses_path, subset="test")

100%|██████████| 40960/40960 [01:01<00:00, 661.08it/s] 
32768it [00:01, 28323.69it/s]


torch.Size([32768, 4, 4])


100%|██████████| 40960/40960 [00:33<00:00, 1230.83it/s]
8192it [00:00, 27742.76it/s]


torch.Size([8192, 4, 4])


100%|██████████| 10240/10240 [00:17<00:00, 578.15it/s]
10240it [00:00, 34821.92it/s]

torch.Size([10240, 4, 4])


In [5]:
if do_training:
    if model_type == 'resnet152':
        model = Resnet152(obj_diameter=obj_diameter, pretrained=use_pretrained).to(device)
    elif model_type == 'resnet18':
        model = Resnet18(obj_diameter=obj_diameter, pretrained=use_pretrained).to(device)
    model.name = model_type
    model = train(model, train_ds, val_ds, lr=0.01, gamma = 0.9, num_epochs=100, device=device, batch_size=32)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]/home/lexi/miniconda3/envs/flowpose/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608853085/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Batches: 100%|██████████| 1024/1024 [05:19<00:00,  3.20it/s]


train loss: 0.18458347888372373
val loss: 0.17918225919129327
Saving model, val loss improved from inf to 0.17918225919129327


Batches: 100%|██████████| 1024/1024 [05:24<00:00,  3.16it/s]


train loss: 0.16309547120908974
val loss: 0.1680671272915788
Saving model, val loss improved from 0.17918225919129327 to 0.1680671272915788


Batches: 100%|██████████| 1024/1024 [05:10<00:00,  3.30it/s]


...

val loss: 0.10678169057064224


Batches: 100%|██████████| 1024/1024 [04:52<00:00,  3.50it/s]


train loss: 0.006923157507344513


Epochs: 100%|██████████| 100/100 [8:42:41<00:00, 313.62s/it]

val loss: 0.10714975230803248


In [6]:
if model_type == 'resnet152':
    model = Resnet152(obj_diameter=obj_diameter).load("saved_models/resnet152_best").to(device)
elif model_type == 'resnet18':
    model = Resnet18(obj_diameter=obj_diameter).load("saved_models/resnet18_best").to(device)

In [16]:
ds_predict = test_ds
predictions = Predictor(model, device).predict_all(ds_predict)

100%|██████████| 10240/10240 [02:57<00:00, 57.59it/s]


In [17]:
transl_thresh = 0.1 # meters
rot_thresh = 10 # degrees
targets = [ds_predict[i][1].to(device) for i in range(len(ds_predict))]
te = transl_error(predictions, targets)
re = rot_error(predictions, targets)
t_recall = recall(te, transl_thresh)
r_recall = recall(re, rot_thresh)

In [18]:
print(f"Mean translational error: {torch.mean(torch.tensor(te))}")
print(f"Mean rotational error: {torch.mean(torch.tensor(re))}")

print(f"Median translational error: {torch.median(torch.tensor(te))}")
print(f"Median rotational error: {torch.median(torch.tensor(re))}")

print(f"Recall for translational error < {transl_thresh}: {t_recall}")
print(f"Recall for rotational error < {rot_thresh}: {r_recall}")

Mean translational error: 0.40934133529663086
Mean rotational error: 63.2928352355957
Median translational error: 0.35013917088508606
Median rotational error: 39.128639221191406
Recall for translational error < 0.1: 0.04482421875
Recall for rotational error < 10: 0.04912109375


In [10]:
# sort entries by a combination of translational and rotational error
sorted_te = sorted(range(len(te)), key=lambda k: te[k]+re[k]/100)
for i in sorted_te:
    print(f"Index: {i}, Translational error: {te[i]}, Rotational error: {re[i]}")
    print(f"Image name: {ds_predict.flow_image_names[i]}")
    print("\n")

Index: 7864, Translational error: 0.018313737586140633, Rotational error: 2.4241432554672273
Image name: shoe-green_viva_sandal_right_5__shoe-asifn_yellow_right_17


Index: 3300, Translational error: 0.038770515471696854, Rotational error: 2.0578544577615534
Image name: shoe-crocs_white_cyan_right_45__shoe-asifn_yellow_right_1


Index: 1300, Translational error: 0.06406048685312271, Rotational error: 2.3189997003474856
Image name: shoe-asifn_yellow_right_47__shoe-crocs_yellow_sandal_right_24


Index: 373, Translational error: 0.039687495678663254, Rotational error: 5.121319488058804
Image name: shoe-asifn_yellow_right_1__shoe-crocs_white_cyan_right_41


Index: 8146, Translational error: 0.053556300699710846, Rotational error: 4.509541580531632
Image name: shoe-green_viva_sandal_right_9__shoe-pink_tiny_crocs_right_6


Index: 4754, Translational error: 0.05111522972583771, Rotational error: 4.857525743093997
Image name: shoe-crocs_yellow_sandal_right_28__shoe-green_viva_sandal_right_42



In [11]:
rgb_images, object_masks, poses, scene_names = load_data(Path("../../output", "dataset_rendered", "shoe"))
meshes = load_meshes_o3d("../../data/housecat6d_meshes/shoe")

Loading meshes...
Loaded 21 meshes.


In [12]:
ResultVisualization(meshes).visualize_results(dataset=ds_predict, rgb_images=rgb_images, poses=poses, predictions=predictions, te=te, re=re)

[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x564e053df8a0 (threading is enabled)
EGL(1.5)
OpenGL(4.1)


100%|██████████| 10240/10240 [33:00<00:00,  5.17it/s]


- different rotation representations (quats), https://github.com/naver/roma
- adaption layer
- only pairs of objects with same distance 
- fixed radius
- maybe polar coordinates?
- maybe other object
- differentiable rendering